1. csv 파일 불러올 때 encoding
 - 한글 encoding : euc-kr, cp949
 - 엑셀로 불러온다음 인코딩을 강제로 해준다.
 - 컬럼때문에 안불러와지면 컬럼 삭제하고 나중에 가져오면된다.
 - 예외처리를해서 euc-kr, cp949 둘다 해본다.
 
 ```
 try
     euc-kr
 except:
     cp949
 ```
 
2. 1번 코로나확진     2번 따릉이
 - 1번과 2번의 기준이되는 index가 똑같아야한다.
 
 - 1번 코로나 확진 데이터 2번 따릉이
 - index   1번 비교데이터       2번 비교데이터
 - 202001      10                      111
 - 202002      321                     323
 - 202003      4                       67

In [3]:
dataset = [
    ['커피','녹차','얼을'], #1인 구매 리스트
    ['커피','얼음','우유'],
    ['커피','딸기'],
    ['바나나','녹차','딸기'],
    ['커피','얼음'],
    ['우유','커피'],
    ['우유','딸기'],
    ['커피'],
    ['얼음','커피']
]

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [5]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
te_ary

array([[ True, False, False,  True, False, False,  True],
       [False, False, False, False,  True,  True,  True],
       [False,  True, False, False, False, False,  True],
       [ True,  True,  True, False, False, False, False],
       [False, False, False, False,  True, False,  True],
       [False, False, False, False, False,  True,  True],
       [False,  True, False, False, False,  True, False],
       [False, False, False, False, False, False,  True],
       [False, False, False, False,  True, False,  True]])

In [6]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,녹차,딸기,바나나,얼을,얼음,우유,커피
0,True,False,False,True,False,False,True
1,False,False,False,False,True,True,True
2,False,True,False,False,False,False,True
3,True,True,True,False,False,False,False
4,False,False,False,False,True,False,True
5,False,False,False,False,False,True,True
6,False,True,False,False,False,True,False
7,False,False,False,False,False,False,True
8,False,False,False,False,True,False,True


In [9]:
f_data = apriori(df, min_support=0.2, use_colnames=True)#지지도
#0.2이상만 보여줘
f_data

,support,itemsets
0,0.222222,(녹차)
1,0.333333,(딸기)
2,0.333333,(얼음)
3,0.333333,(우유)
4,0.777778,(커피)
5,0.333333,"(커피, 얼음)"
6,0.222222,"(우유, 커피)"


In [13]:
from mlxtend.frequent_patterns import association_rules
#지지도(support) 전체거래항목중 a,b를 포함하는 비율
#신뢰도(confidence) a를 포함하는 거래중에 a,b가 동시에 거래되는 비중
#향상도(Lift) a와 b사이에 상호관계가 없으면 1 //// 1보다 큰값 연관성이 낮다

#threshold = 신뢰도

# association_rules(f_data, metric='confidence', min_threshold= 0.3)
association_rules(f_data, metric='lift', min_threshold= 0.3)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(커피),(얼음),0.777778,0.333333,0.333333,0.428571,1.285714,0.074074,1.166667
1,(얼음),(커피),0.333333,0.777778,0.333333,1.000000,1.285714,0.074074,inf
2,(우유),(커피),0.333333,0.777778,0.222222,0.666667,0.857143,-0.037037,0.666667
3,(커피),(우유),0.777778,0.333333,0.222222,0.285714,0.857143,-0.037037,0.933333
